# Example notebook for analysing event and cluster data from Krakow Test Beam
+ I give some example histograms with various cuts and stuff
+ Important was to establish the procedure for getting the corryvreckan data and looping over the trees

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-1">Import libraries</a></span></li><li><span><a href="#Define-custom-cpp-functions" data-toc-modified-id="Define-custom-cpp-functions-2">Define custom cpp functions</a></span></li><li><span><a href="#Custom-classes" data-toc-modified-id="Custom-classes-3">Custom classes</a></span></li><li><span><a href="#Read-root-file-and-convert-to-python" data-toc-modified-id="Read-root-file-and-convert-to-python-4">Read root file and convert to python</a></span></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-5">Analysis</a></span><ul class="toc-item"><li><span><a href="#Cluster-size-(also-column-width-/-row-width-of-clusters)" data-toc-modified-id="Cluster-size-(also-column-width-/-row-width-of-clusters)-5.1">Cluster size (also column width / row width of clusters)</a></span></li><li><span><a href="#Number-of-clusters" data-toc-modified-id="Number-of-clusters-5.2">Number of clusters</a></span></li></ul></li></ul></div>

## Import libraries

In [1]:
from ipywidgets import IntProgress
from IPython.display import display,HTML
import time
import ROOT
ROOT.gSystem.Load("/home/berki/Software/corryvreckan/lib/libCorryvreckanObjects.so")

display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

Welcome to JupyROOT 6.26/10


## Define custom cpp functions
+ Since some functions in corryvreckan (e.g. Cluster.global()) have names which are not allowed in python we need these cell magic to be able extract the info into python

In [2]:
%%cpp
std::vector<double> getGlobal(corryvreckan::Cluster* cluster) {
    double x,y,z;
    cluster->global().GetCoordinates(x,y,z);
    std::vector<double> out{x,y,z};
    return out;
}

In [3]:
%%cpp
std::vector<double> getLocal(corryvreckan::Cluster* cluster) {
    double x,y,z;
    cluster->local().GetCoordinates(x,y,z);
    std::vector<double> out{x,y,z};
    return out;
}

## Custom classes
+ Customized python classes for events and clusters
+ `To-Do`: don't define these here in the notebook but outside and jsut import them

In [4]:
class Event:
    def __init__(self):
        self.timestamp = 0
        self.clusters = []
    def setTimestamp(self,time):
        self.timestamp = time
    def addCluster(self,cluster):
        self.clusters.append(cluster)
    def getNClusters(self):
        return len(self.clusters)
    
class Cluster:
    def __init__(self):
        self.detector = 0
        self.size = 0
        self.col = 0
        self.row = 0
        self.charge = 0
        self.localPos = [0,0]
        self.globalPos = [0,0,0]
        self.split = False
        self.colWidth = 0
        self.rowWidth = 0
    def setData(self,corryCluster):
        self.detector = corryCluster.getDetectorID()
        self.size = corryCluster.size()
        self.col = corryCluster.column()
        self.row = corryCluster.row()
        self.charge = corryCluster.charge()
        self.localPos = ROOT.getLocal(corryCluster)
        self.globalPos = ROOT.getGlobal(corryCluster)
        self.split = corryCluster.isSplit()
        self.colWidth = corryCluster.columnWidth()
        self.rowWidth = corryCluster.rowWidth()


## Read root file and convert to python
+ Converting to python classes is actually not needed in this example and we could very well just take the corryvreckan cluster objects for filling histograms
+ However in the long run we will probably need to add our own stuff to clusters so I already do the conversion here

In [5]:
filePath = "~/Software/KrakowTestBeamAnalysis/output/run456060805_221112060811_clusters.root"
file = ROOT.TFile.Open(filePath)
tEvents = file.Get("Event")
tClusters = file.Get("Cluster")

#tClusters.GetListOfBranches().ls() # list detectors
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]

events = []
#nEvents = tClusters.GetEntriesFast() # to read full file

nEvents = 10000
#progress bar
f = IntProgress(min=0, max=nEvents, description="Converting:")
display(f)

for index,event in enumerate(tClusters):
    if index > nEvents : break #to limit the number of events to be processed
    myEvent = Event()
    for detector in detectors:
        branch = getattr(event, detector)
        for cluster in branch:
            myClus = Cluster()
            myClus.setData(cluster)
            myEvent.addCluster(myClus)
    events.append(myEvent)
    f.value += 1
    
f.bar_style = "success"

IntProgress(value=0, description='Converting:', max=10000)

## Analysis

### Cluster size (also column width / row width of clusters)
+ Since we observe that the inner layers have much more hits, we know they must be from delta electrons or from the beam etc.
+ In order to get rid of them, one idea would be to apply cuts on cluster size as the wanted proton hits should lead to larger clusters

In [6]:
%jsroot on
canvas = ROOT.TCanvas()
legend = ROOT.TLegend()

# Calculate histogram
hClusSize = [ROOT.TH1F("Cluster_Size_"+det, "Size of cluster - "+det, 30, 0, 30) for det in detectors]
for event in events:
    for index,detector in enumerate(detectors):
        for cluster in event.clusters:
            if cluster.detector == detector:
                hClusSize[index].Fill(cluster.size)

#Draw histogram
for index,hist in enumerate(hClusSize):
    hist.Scale(1/hist.Integral())
    hist.SetMaximum(0.15)
    hist.Draw("SAME L")
    hist.SetLineColor(index+1)
    hist.SetLineWidth(2)
    hist.SetXTitle("Cluster size [pixels]")
    hist.SetYTitle("Counts (normalized to integral)")
    legend.AddEntry(hist,detectors[index])

legend.Draw() 
canvas.Draw()

In [7]:
%jsroot on
canvas = ROOT.TCanvas()
canvas.SetCanvasSize(900,600)

# Calculate histogram
h2DClusSizeVsColWidth = [ROOT.TH2F("Cluster_Size_vs_ColWidth"+det, "Size of cluster vs column width- "+det, 30,0,30,10,0,10) for det in detectors]
for event in events:
    for index,detector in enumerate(detectors):
        for cluster in event.clusters:
            if cluster.detector == detector:
                h2DClusSizeVsColWidth[index].Fill(cluster.size,cluster.colWidth)

#Draw histogram
canvas.Divide(3,2)
for pad in range(5):
    canvas.cd(pad+1)
    h2DClusSizeVsColWidth[pad].Draw("COLZ")
    h2DClusSizeVsColWidth[pad].SetTitle("ALPIDE_"+str(pad)+"; Cluster Size [pixel]; Cluster column width [pixel]")

canvas.Draw()

### Number of clusters

In [8]:
%jsroot on
canvas = ROOT.TCanvas()
canvas.SetCanvasSize(900,900)
canvas.Divide(2,2)
cuts = {0:0,
        1:5,
        2:10,
        3:15
       }
hNClusDet = []
for pad in range(4):
    # Fill histograms
    hNClusDet.append([ROOT.TH1F("nClusters_"+det+"_cut_"+str(pad), "nClusters of events - "+det, 30, 0, 30) for det in detectors])
    for event in events:
        for index,detector in enumerate(detectors):
            count = 0
            for cluster in event.clusters:
                if cluster.detector == detector and cluster.size > cuts.get(pad):
                    count += 1
            if count:
                hNClusDet[pad][index].Fill(count)
                
    # Draw histograms
    canvas.cd(pad+1)
    legend = ROOT.TLegend()
    #hNClus.Draw()
    for index,hist in enumerate(hNClusDet[pad]):
        hist.Scale(1/hist.Integral())
        #hist.SetMaximum(0.15)
        hist.SetLineColor(index+1)
        hist.SetLineWidth(2)
        hist.SetXTitle("# of clusters per event")
        hist.SetYTitle("Counts (normalized to integral)")
        hist.SetTitle("Cluster size > "+str(cuts.get(pad)))
        legend.AddEntry(hist,detectors[index])
        hist.Draw("SAME HIST")
    legend.Draw()
canvas.Draw()